In [1]:
import sys

!{sys.executable} -m pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 8.5 MB/s eta 0:00:00


In [2]:
HOME_DIR = "/home_remote"
HOME = "/home/thi.tra.my.nguyen"

In [3]:
from liwc import Liwc
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
# Load LIWC dictionary
liwc = Liwc(os.path.join(HOME_DIR, "master_thesis/LIWC2007_English100131.dic"))

In [76]:
# Load data
%store -r df
%store -r df_neg

In [77]:
#combine all text of same user into one string
text_per_user_pos = df.groupby('User')['Text'].apply(' '.join).reset_index()
text_per_user_neg = df_neg.groupby('User')['Text'].apply(' '.join).reset_index()
#add label
text_per_user_pos['label'] = 1
text_per_user_neg['label'] = 0

In [ ]:
# BACKUP
#combine Text and Title
df['Text_Title'] = df['Title'] + '.' + df['Text']
df_neg['Text_Title'] = df_neg['Title'] + '.' + df_neg['Text']
#combine all text of same user into one string
text_per_user_pos = df.groupby('User')['Text_Title'].apply(' '.join).reset_index()
text_per_user_neg = df_neg.groupby('User')['Text_Title'].apply(' '.join).reset_index()
#add label
text_per_user_pos['label'] = 1
text_per_user_neg['label'] = 0

In [81]:
#input first user of positive to liwc
temp = liwc.parse(text_per_user_pos['Text'][0].split(' '))
#sort temp descending
temp = sorted(temp.items(), key=lambda x: x[1], reverse=True)

In [25]:
# input text to LIWC
liwc_pos = [liwc.parse(text.split(' ')) for text in text_per_user_pos['Text']]
liwc_neg = [liwc.parse(text.split(' ')) for text in text_per_user_neg['Text']]
#then buid a dictionary with key as user, value as a list of LIWC features
liwc_pos_dict = {}
liwc_neg_dict = {}
for i in range(len(liwc_pos)):
    liwc_pos_dict[text_per_user_pos['User'][i]] = liwc_pos[i]
    liwc_neg_dict[text_per_user_neg['User'][i]] = liwc_neg[i]

In [ ]:
# BACKUP
# input text to LIWC
liwc_pos = [liwc.parse(text.split(' ')) for text in text_per_user_pos['Text_Title']]
liwc_neg = [liwc.parse(text.split(' ')) for text in text_per_user_neg['Text_Title']]
#then buid a dictionary with key as user, value as a list of LIWC features
liwc_pos_dict = {}
liwc_neg_dict = {}
for i in range(len(liwc_pos)):
    liwc_pos_dict[text_per_user_pos['User'][i]] = liwc_pos[i]
for i in range(len(liwc_neg)):
    liwc_neg_dict[text_per_user_neg['User'][i]] = liwc_neg[i]

In [29]:
#calculate percentage of each LIWC feature per user
liwc_pos_percent = {}
liwc_neg_percent = {}
for key, value in liwc_pos_dict.items():
    liwc_pos_percent[key] = {k: v / sum(value.values()) for k, v in value.items()}
for key, value in liwc_neg_dict.items():
    liwc_neg_percent[key] = {k: v / sum(value.values()) for k, v in value.items()}


In [38]:
#correlation of LIWC features with label
liwc_pos_percent_df = pd.DataFrame(liwc_pos_percent).T
liwc_neg_percent_df = pd.DataFrame(liwc_neg_percent).T
liwc_pos_percent_df['label'] = 1
liwc_neg_percent_df['label'] = 0
liwc_percent_df = pd.concat([liwc_pos_percent_df, liwc_neg_percent_df])
liwc_percent_df = liwc_percent_df.fillna(0)
corr = liwc_percent_df.corr()



In [40]:
#choose the top 15 most correlated features to label
corr_label = corr['label'].sort_values(ascending=False)
corr_label = corr_label[1:16]
corr_label

i          0.360488
anx        0.358541
health     0.348380
sad        0.345127
affect     0.321898
negemo     0.320290
feel       0.278152
ppron      0.272873
adverb     0.249147
pronoun    0.248551
insight    0.213088
excl       0.205903
discrep    0.204839
cogmech    0.203785
filler     0.198220
Name: label, dtype: float64

### LIWCalike

In [34]:
with open('/home_remote/dic_avg100_annotated_official.xlsx', 'rb') as f:
    liwc2 = pd.read_excel(f)

In [35]:

liwc2['Count'] = liwc2['Term'].str.split().str.len()


In [36]:
liwc2

,Unnamed: 0,Category,Term,Count
0,0.0,Pronouns,"['both', 'somebody', 'naught', 'whoever', 'eve...",124
1,NaN,I,"['mine', 'my', 'me', 'I', 'myself']",5
2,NaN,We,"['our', 'us', 'we', 'ours', 'ourselves']",5
3,NaN,You,"['yourself', 'thine', 'thyself', 'your', 'yon'...",13
4,NaN,Shehe,"['her', 'hers', 'him', 'she', 'his', 'herself'...",11
5,NaN,They,"['their', 'theirs', 'them', 'they', 'themselve...",7
6,NaN,ipron,"['both', 'everything', 'somebody', 'they', 'fe...",48
7,1.0,Articles,"['an', 'the', 'a']",3
8,NaN,auxverb,"['should', 'is', 'could', ""amn't"", 'will', 'mi...",29
9,2.0,Past tense,"['had', 'were', 'went', 'was', 'walked', 'came...",158


In [30]:
def main(input_, dic):
    dic['Terms'] = dic['Term'].apply(lambda x: ast.literal_eval(x))
    result = dict(zip(dic['Category'], dic['Terms']))
    lexicon = parse_dict(result)
    parse, category_names = load_token_parser(result)
    
    input_tokens = tokenize(input_.lower())
    counts = Counter(category for token in input_tokens for category in parse(token))
    
    a = pd.DataFrame.from_dict(dict(counts),orient = 'index').reset_index()
    a= a.rename(columns={"index": "Category", 0: "Count"})
    a['Percentage(%)']= round(a['Count']/len(input_.split(' '))*100,2)
    return a

5817